In [2]:
import pandas as pd
from requests import get

positions = {1:'QB',2:'RB',3:'WR',4:'TE',5:'K',16:'D/ST'}
teams = {1:'ATL',2:'BUF',3:'CHI',4:'CIN',5:'CLE',
        6:'DAL', 7:'DEN',8:'DET',9:'GB',10:'TEN',
        11:'IND',12:'KC',13:'OAK',14:'LAR',15:'MIA',
        16:'MIN',17:'NE',18:'NO',19:'NYG',20:'NYJ',
        21:'PHI',22:'ARI',23:'PIT',24:'LAC',25:'SF',
        26:'SEA',27:'TB',28:'WAS',29:'CAR',30:'JAX',
        33:'BAL',34:'HOU'}
projections = []
actuals = []
for season in [2018,2020]:
    url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + str(season)
    url = url + '/segments/0/leaguedefaults/3?scoringPeriodId=1&view=kona_player_info'
    players = get(url).json()['players']
    for player in players:
        stats = player['player']['stats']
        for stat in stats:
            c1 = stat['seasonId'] == season
            c2 = stat['statSplitTypeId'] == 1
            c3 = player['player']['defaultPositionId'] in positions
            if (c1 and c2 and c3):
                data = {
                    'Season':season,
                    'PlayerID':player['id'],
                    'Player':player['player']['fullName'],
                    'Position':positions[player['player']['defaultPositionId']],
                    'Week':stat['scoringPeriodId']}
                if stat['statSourceId'] == 0:
                    data['Actual Score'] = stat['appliedTotal']
                    data['Team'] = teams[stat['proTeamId']]
                    actuals.append(data)
                else:
                    data['Projected Score'] = stat['appliedTotal']
                    projections.append(data)         
actual_df = pd.DataFrame(actuals)
proj_df = pd.DataFrame(projections)
df = actual_df.merge(proj_df, how='inner', on=['PlayerID','Week','Season'], suffixes=('','_proj'))
df = df[['Season','Week','PlayerID','Player','Team','Position','Actual Score','Projected Score']]
f_path = 'C:/Users/Someone/Documents/something.csv'
df.to_csv(f_path, index=False)

SSLError: HTTPSConnectionPool(host='fantasy.espn.com', port=443): Max retries exceeded with url: /apis/v3/games/ffl/seasons/2018/segments/0/leaguedefaults/3?scoringPeriodId=1&view=kona_player_info (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1108)')))